In [486]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np
import copy
import itertools


# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit



In [495]:
def gen_angles(x):
    
    if len(x)>1:
        new_x = np.zeros(int(len(x)/2))
        
        for k in range(len(new_x)):
            new_x[k] = np.sqrt(x[2*k]**2 + x[2*k+1]**2)
        
        inner_angles = gen_angles(new_x)
                
        angles = np.zeros(int(len(x)/2))

        for k in range(len(new_x)):
            if(new_x[k]!=0):
                if(x[2*k]>0):
                    angles[k] = 2*np.arcsin(x[2*k+1]/new_x[k])
                else:
                    angles[k] = 2*np.pi - 2*np.arcsin(x[2*k+1]/new_x[k])
            else:
                angles[k] = 0
        
        angles = np.concatenate((inner_angles, angles))
        
        return np.array(angles)
    
    else: return np.ones(0)
    
    
    
n_0 = 20  # Number of elements in the array
array = [0] * n_0  # Initialize the array with zeros

# Calculate the cumulative sums of powers of 2
for i in range(n_0):
    if i == 0:
        array[i] = 1
    else:
        array[i] = array[i-1] + 2 ** i
array = np.array(array)-1



def level(k):
    
    for i in range(len(array)):
        
        if(k==0): return 0
        
        if (k>array[i] and k<=array[i+1]): return i+1
    

levels = [level(np.arange(0,N)[i]) for i in range(N)]

def parity(level):
    
    n = level
    combinations = list(itertools.product([0, 1], repeat=n))
    return (combinations)
    
def gen_circuit(qubits, angles): #recive a list of n qubits
    
    circuit = cirq.Circuit()
    counter = 0

    for k in range(len(angles)):
        
        j = levels[k]
        
        if(j!=levels[k-1] and k>0):
            counter = 0
        
        p = parity(j)
        op = cirq.ry(angles[k]).on(qubits[j]).controlled_by(*qubits[0:j], control_values=parity(j)[counter])
        #for i in range(j):
         #   op = op.controlled_by(qubits[j-1-i], control_values=[1])
        
        counter += 1
        circuit += op

    return circuit

In [496]:
def amplitude_encoding(qubits, data, print_it=False):
    
    """ Make amplitude encoding. Return the circuit """
    
    N = len(data) #dimension of the vector
    n = np.log2(N) #number of qubits
    
    angles = np.hstack(gen_angles(data)) #compute the angles to do amp encoding

    circuit = gen_circuit(qubits, angles) #make the circuit
    
    if (print_it==True):
        SVGCircuit(circuit)
        
    return circuit

In [494]:
bits = cirq.GridQubit.rect(1, 6)

amplitude_encoding(bits, np.ones(64), print_it=True)



(0, 0): ───Ry(0.5π)───(0)────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────
                      │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │
(0, 1): ──────────────Ry(0.5π)───Ry(0.5π)───(0)────────@──────────(0)────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────@──────────
                                            │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │
(0, 2): ────────────────────────────────────Ry(0.5π)───Ry(0.5π)───Ry(0.5π)───Ry(0.5π)───(0)────────@──────────(0)────────@──────────(0)────────@──────────(0)────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────(0)────────(0)────────(0)────────(0)────────@──────────@──────────@──────────@──────────
                                                                                        │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          │          